<a href="https://colab.research.google.com/github/1013115/KTB_Project/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 협업기반 필터링

In [3]:
import pandas as pd
import ast

# 데이터 로드 및 전처리
file_path = 'result (2).csv'
data = pd.read_csv(file_path)
data['labels'] = data['labels'].apply(ast.literal_eval)
data['scores'] = data['scores'].apply(ast.literal_eval)

def recommend_content_based(preferences, top_n=5):
    recommendations = []

    for i, row in data.iterrows():
        relevance_score = sum(row['scores'][row['labels'].index(pref)] for pref in preferences if pref in row['labels'])
        recommendations.append((row['sequence'], relevance_score))

    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]

# 예시 사용자 선호도
user_preferences = ['가족', '자연']
top_content_based = recommend_content_based(user_preferences)

for i, (description, score) in enumerate(top_content_based, 1):
    print(f"[콘텐츠 기반] {i}. {description} (점수: {score:.2f})")


[콘텐츠 기반] 1. 국립산림과학원 남부산림자원연구소는 [자원과 환경]이라는 주제로 전시된 자료를 통해, 우리 생활과 밀접한 관계를 맺고 있는 임업과 산림을 올바르게 이해할 수 있는 국민 학습장의 역할을 다하고, 산업화, 도시화로 점점 메말라가는 사람들의 정서를 함양시키는 데 일익을 담당하는 동시에 산림과 관련된 자료의 수집, 보전 및 전시를 목적으로 한다.
남부산림자원연구소는 남부지역 산림자원의 다원적 기능 발굴 및 증진을 위한 산림과학기술력 확보와 건전한 산림육성을 목표로 단기소득 및 특수임산자원의 증식과 고부가가치 이용 기술 개발, 남부산림자원의 종합관리기술 개발 연구를 추진하고 있으며, 아울러 소나무재선충병정밀진단 및 방제에 대한 현장 적용 실용화 연구를 강화하면서 산림과 특수임산자원의 병해충 종합관리 기술개발에 대한 연구를 수행하고 있다. (점수: 0.87)
[콘텐츠 기반] 2. 강화자연사박물관은 산업화 도시화로 오염되어 가는 환경에 대한 소중함을 일깨우고 자연과 생명에 대한 올바른 인식을 세우는데 도움이 되고자 2015년에 개관하였다. 강화자연사박물관은 수집 및 연구, 전시를 통해 자연의 신비를 체험하고 학습할 수 있는 공간으로,  우리가 사는 지구가 속한 태양계의 탄생에서 지구를 구성하고 있는 다양한 광물, 생물들을 각각의 특별한 주제로 나누어 전시하고 있다. 이를 통해 박물관을 방문하는 관람객들이 자연보존과 생명의 중요성을 다시 한번 생각하게 하며, 자연과학에 대해 이해를 높일 수 있도록 한다. 세계문화유산인 강화고인돌유적지 부근에 위치해 있어 고인돌과 함께 관람하기 좋다. (점수: 0.86)
[콘텐츠 기반] 3. 천문우주관련 전문과학관으로서 포천 아트밸리내에 위치하여 조각 공원과 폐채석장을 활용한 아름다운 자연경관이 자랑이다. 매시간 천체투영실에서 별자리 설명 및 특수영상 등의 체험과 주간에는 태양관측, 야간에는 달과 행성등의 천체관측을 운영한다. 주말과학체험교실과 과학강연 등 다양한 과학체험프로그램도 운영중이다. (점수: 0.78)
[콘텐츠 

## 사용자 기반 필터링

이의 경우 사용자 클릭수가 있거나.. 사용자의 평가가 존재해야할 듯

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 가상의 사용자-아이템 행렬 생성
# 사용자 선호도 예시 데이터 (사용자 ID, 여행지 ID, 점수)
user_ratings = {
    'user1': {'0': 4, '1': 3, '2': 5, '3': 4, '4': 2},
    'user2': {'0': 3, '1': 4, '2': 2, '3': 5, '4': 4},
    'user3': {'0': 5, '1': 2, '2': 4, '3': 3, '4': 3},
}

# 사용자-아이템 행렬 구성
user_item_matrix = pd.DataFrame(user_ratings).T.fillna(0)

# 사용자 유사도 계산 (코사인 유사도)
user_similarity = cosine_similarity(user_item_matrix)

# 예시 사용자 선호도
target_user = 'user1'
similar_users = user_similarity[user_item_matrix.index.get_loc(target_user)]

# 비슷한 사용자들의 평점 평균을 기반으로 추천
def recommend_user_based(target_user, top_n=5):
    target_index = user_item_matrix.index.get_loc(target_user)
    similar_users = user_similarity[target_index]
    weighted_sum = np.dot(similar_users, user_item_matrix)
    recommendation_scores = weighted_sum / (np.array([np.abs(similar_users).sum(axis=0)]) + 1e-9)

    recommendations = []
    for idx, score in enumerate(recommendation_scores):
        if user_item_matrix[target_user][idx] == 0:  # 아직 평가하지 않은 아이템만 추천
            recommendations.append((idx, score))

    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]

top_user_based = recommend_user_based('user1')

for i, (index, score) in enumerate(top_user_based, 1):
    print(f"[사용자 기반 협업 필터링] {i}. 여행지 {index} (예상 점수: {score:.2f})")


## 하이브리드

이것은 사용자 기반 필터링이 해결되어야..

In [10]:
def recommend_hybrid(preferences, target_user, top_n=5):
    # 콘텐츠 기반 추천
    content_based_recommendations = recommend_content_based(preferences, top_n)

    # 사용자 기반 추천
    user_based_recommendations = recommend_user_based(target_user, top_n)

    # 결합 추천
    combined_recommendations = content_based_recommendations + user_based_recommendations
    unique_recommendations = {rec[0]: rec[1] for rec in combined_recommendations}
    sorted_recommendations = sorted(unique_recommendations.items(), key=lambda x: x[1], reverse=True)

    return sorted_recommendations[:top_n]

top_hybrid = recommend_hybrid(user_preferences, 'user1')

for i, (index, score) in enumerate(top_hybrid, 1):
    print(f"[하이브리드] {i}. 여행지 {index} (점수: {score:.2f})")


[KNN 기반] 1. 재단법인 부산문화회관이 운영하는 부산문화회관은 1988년 대극장 준공을 시작으로 1993년 전관을 개관하였다. 대극장(1,417석)과 중극장(783석), 어린이 전용극장인 사랑채극장(312석)과 클래식 전용극장인 챔버홀(414석)을 운영하고 있으며, 뮤지컬, 클래식, 무용, 연극 등 다양한 공연을 개최하고 있다.
전시실에서는 부산미술대전, 부산서예대전 등 대규모 공모전을 비롯하여 다양한 기획전시가, 2024년 신규 개관한 배움터에서는 매년 상·하반기 문화예술 아카데미가 열린다.
야외공원은 부산문화회관과 부산시립박물관, UN공원을 하나로 이어주는 문화공원으로서 시민들을 위한 휴식 공원으로 조성되어 유치원, 어린이들의 휴식공간, 소풍장소로 각광받고 있다. (거리: 2.83)
[KNN 기반] 2. ※ 해양박물관 리모델링 공사 관계로 한시적으로 운영 중단됩니다.</br>
화진포 관광지구 내에 위치한 해양박물관으로 세계적으로 희귀한 각종 조개류, 갑각류, 산호류, 화석류, 박제 등 다양한 수중생물들을 볼 수 있는 패류박물관과 어류전시관으로 구성되어 있다. 1층은 어류전시관으로 180도 머리 위를 지나는 물고기들을 관람할 수 있고, 2층은 오션 비치, 동해 바다, 무척추 동물관과 입체영상관을 볼 수 있다. 특히 2층에서 상영하는 입체 영상을 통해 화진포의 호수와 바닷속 생태계를 공부할 수 있다. 고성군의 명물인 명태를 비롯해 동해 바다 돌고래, 고성 대진항, 거진항 어부들의 고기잡이 과정도 살펴볼 수 있다. (거리: 2.83)
[KNN 기반] 3. 1996년 개관된 양구통일관은 민통선 최북단 양구군 해안면에 위치하고 있다. 통일에 대비해 북한의 현실을 이해하고 통일의식을 고취하기 위한 교육장 역할을 하기 위해 펀치볼분지 북단 제4땅굴 앞 5km 지점에 건립되었다. 통일관은 제1,2 전시실, 농특산물 판매장으로 나뉜다. 북한의 실상을 알 수 있는 생활용품, 수출품과 사진 등을 상설 전시하고 있으며, 농특산물 전시 판매장에서는 농특산물과 관광기념품 등을

## KNN

아마 사용자기반 필터링에서 사용할듯

In [12]:
import pandas as pd
import ast
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 데이터 로드 및 전처리
file_path = 'result (2).csv'
data = pd.read_csv(file_path)
data['labels'] = data['labels'].apply(ast.literal_eval)
data['scores'] = data['scores'].apply(ast.literal_eval)

# 데이터 인덱스를 재설정하여 순차적인 인덱스를 가지도록 함
data.reset_index(drop=True, inplace=True)

# 라벨을 바이너리 벡터로 변환
all_labels = ['가족', '예술', '연인', '친구', '역사', '자연', '체험', '바다', '산']
label_vectors = []
for labels in data['labels']:
    vector = [1 if label in labels else 0 for label in all_labels]
    label_vectors.append(vector)

label_vectors = np.array(label_vectors)

# KNN 모델 훈련
knn = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(label_vectors)

def recommend_knn(preferences, top_n=5):
    # 사용자 선호도를 바이너리 벡터로 변환
    user_vector = np.array([[1 if label in preferences else 0 for label in all_labels]])

    # KNN을 사용하여 가장 가까운 이웃 찾기
    distances, indices = knn.kneighbors(user_vector, n_neighbors=top_n)

    recommendations = []
    for idx in indices[0]:
        recommendations.append((data.iloc[idx]['sequence'], distances[0][indices[0].tolist().index(idx)]))

    return recommendations

# 예시 사용자 선호도
user_preferences_1 = ['가족', '자연', '체험']
top_knn_recommendations_1 = recommend_knn(user_preferences_1)

print("사용자 선호도: ", user_preferences_1)
for i, (description, distance) in enumerate(top_knn_recommendations_1, 1):
    print(f"[KNN 기반] {i}. {description} (거리: {distance:.2f})")



사용자 선호도:  ['가족', '자연', '체험']
[KNN 기반] 1. 재단법인 부산문화회관이 운영하는 부산문화회관은 1988년 대극장 준공을 시작으로 1993년 전관을 개관하였다. 대극장(1,417석)과 중극장(783석), 어린이 전용극장인 사랑채극장(312석)과 클래식 전용극장인 챔버홀(414석)을 운영하고 있으며, 뮤지컬, 클래식, 무용, 연극 등 다양한 공연을 개최하고 있다.
전시실에서는 부산미술대전, 부산서예대전 등 대규모 공모전을 비롯하여 다양한 기획전시가, 2024년 신규 개관한 배움터에서는 매년 상·하반기 문화예술 아카데미가 열린다.
야외공원은 부산문화회관과 부산시립박물관, UN공원을 하나로 이어주는 문화공원으로서 시민들을 위한 휴식 공원으로 조성되어 유치원, 어린이들의 휴식공간, 소풍장소로 각광받고 있다. (거리: 2.45)
[KNN 기반] 2. ※ 해양박물관 리모델링 공사 관계로 한시적으로 운영 중단됩니다.</br>
화진포 관광지구 내에 위치한 해양박물관으로 세계적으로 희귀한 각종 조개류, 갑각류, 산호류, 화석류, 박제 등 다양한 수중생물들을 볼 수 있는 패류박물관과 어류전시관으로 구성되어 있다. 1층은 어류전시관으로 180도 머리 위를 지나는 물고기들을 관람할 수 있고, 2층은 오션 비치, 동해 바다, 무척추 동물관과 입체영상관을 볼 수 있다. 특히 2층에서 상영하는 입체 영상을 통해 화진포의 호수와 바닷속 생태계를 공부할 수 있다. 고성군의 명물인 명태를 비롯해 동해 바다 돌고래, 고성 대진항, 거진항 어부들의 고기잡이 과정도 살펴볼 수 있다. (거리: 2.45)
[KNN 기반] 3. 1996년 개관된 양구통일관은 민통선 최북단 양구군 해안면에 위치하고 있다. 통일에 대비해 북한의 현실을 이해하고 통일의식을 고취하기 위한 교육장 역할을 하기 위해 펀치볼분지 북단 제4땅굴 앞 5km 지점에 건립되었다. 통일관은 제1,2 전시실, 농특산물 판매장으로 나뉜다. 북한의 실상을 알 수 있는 생활용품, 수출품과 사진 등을 상설 전시하고 있으며, 

## TF-IDF 협업기반 필터링

In [16]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 로드 및 전처리
file_path = 'result (2).csv'
data = pd.read_csv(file_path)
data['labels'] = data['labels'].apply(ast.literal_eval)
data['scores'] = data['scores'].apply(ast.literal_eval)

# 여행지 설명을 텍스트로 추출
descriptions = data['sequence'].values

korean_stop_words = ["의", "가", "이", "은", "는", "을", "를", "에", "과", "와", "한", "하다"]


# TF-IDF 벡터라이저를 사용하여 설명을 벡터화
tfidf_vectorizer = TfidfVectorizer(stop_words=korean_stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(descriptions)

# TF-IDF를 기반으로 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def recommend_tfidf_based(index, top_n=5):
    # 특정 여행지와 유사한 여행지의 유사도 점수 가져오기
    sim_scores = list(enumerate(cosine_sim[index]))

    # 유사도 점수를 기준으로 정렬하여 상위 N개 여행지 선택
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 상위 N개 추천 여행지 반환 (자기 자신 제외)
    top_similar_indices = [i[0] for i in sim_scores[1:top_n+1]]

    recommendations = data.iloc[top_similar_indices]
    return recommendations

# 예시로 첫 번째 여행지와 유사한 여행지 추천
index = 2
top_tfidf_recommendations = recommend_tfidf_based(index)

print("원본 여행지 설명: ", data.iloc[index]['sequence'])
print("\n추천 여행지:\n")
for i, row in top_tfidf_recommendations.iterrows():
    print(f"{i+1}. {row['sequence']}")


원본 여행지 설명:  폐산업시설이었던 폐수처리장을 리모델링한 곳으로 수원산업단지 내에 위치해 있다. 뉴지엄이란 ‘뉴(New)와 뮤지엄(Museum)’의 합성어로 생활에 예술을 더하는 복합문화공간이라는 뜻이다. 전시실은 고색뉴지엄 전시 및 각종 행사가 이루어지는 시설로 폐수처리장의 흔적을 느낄 수 있는 고색뉴지엄의 대표 공간이다. 교육실은 문화 강좌 및 교육 프로그램을 수강하는 곳이며 회의나 세미나 등 다양한 용도로 활용되는 다목적 시설이다. 이외에도 문화 관련 서적 및 잡지를 열람할 수 있는 아카이브가 있다. 또한 주변에 황구지천 산책로가 위치해 있어 자연과 공존된 분위기를 함께 느낄 수 있다.

추천 여행지:

114. 완주향토예술문화회관은 완주군 삼례읍에 있는 문화시설로 전시장과 공연장이 있다.
이곳은 주민이 중심이 되어 문화예술 활동을 할 수 있는 공간으로 다양한 문화 체험 기회를 제공한다. 지역주민이 언제든지 자유롭게 일상에서 이용할 수 있는 복합적인 문화 활동 공간이다. 공연 외에도 전시, 문화예술교육 등 다양한 프로그램을 통해 지역민들에게 있어 일상 속의 문화 동반자 역할을 하고 있다. 이를 위해 전시실은 전문 작가뿐만 아니라 남녀노소 누구나 예술적 역량을 펼칠 수 있는 열린 문화공간으로 운영하고 있다. 공연장, 전시실 외에도 전면 거울이 있거나 방음이 되는 연습실, 세미나실 등 다양한 시설을 대관하여 이용할 수 있다.
주변에는 삼례신금공원, 완주군 공설운동장, 완주군 청소년수련관 등이 있어 연계하여 둘러보거나 이용할 수 있다.
471. 부산 센텀에 위치한 뮤지엄 원은 예술 전문 기획사 쿤스트원이 2019년에 설립한 미디어 전문 현대 미술관으로, 2022년 3월 뮤지엄 원이라는 새로운 이름으로 재개관 했다. 약 700여 평 규모의 복층 형태로 이루어진 뮤지엄 원은 8천만 개의 LED를 바닥과 천장, 벽면에 설치하여 관람객들을 예술 작품 속으로 안내하는 초현실적인 경험을 선사한다. 
뮤지엄 원은 대중을 위한 예술, 대중이 체험하고 즐길 수 있는 예술을